In [80]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [81]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

In [82]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train.sample(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
494,0533_01,Mars,False,D/21/P,TRAPPIST-1e,33.0,False,20.0,0.0,384.0,557.0,0.0,Tops Stike,False
495,0534_01,Europa,False,B/23/P,TRAPPIST-1e,34.0,False,98.0,715.0,29.0,5732.0,1902.0,Asellux Parbage,False
6744,7114_01,Earth,False,F/1362/S,TRAPPIST-1e,24.0,False,0.0,489.0,21.0,0.0,184.0,Camiet Ricklaney,False
7720,8237_01,Earth,False,F/1577/S,TRAPPIST-1e,19.0,False,511.0,0.0,0.0,267.0,0.0,Oraryn Bakerson,False
2994,3237_03,Mars,False,D/104/P,TRAPPIST-1e,22.0,False,2646.0,0.0,599.0,117.0,0.0,Mon Lapie,False


In [83]:
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test.sample(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
3240,7112_01,Earth,False,F/1360/S,TRAPPIST-1e,22.0,False,22.0,0.0,1367.0,0.0,0.0,Pamont Blange
1414,3017_01,Earth,False,G/491/P,TRAPPIST-1e,17.0,False,0.0,4738.0,188.0,0.0,NaN,Therly Wilsoney
2008,4322_01,Earth,True,G/712/P,55 Cancri e,16.0,False,0.0,0.0,0.0,0.0,0.0,Melara Ballocks
174,0354_01,Earth,False,F/79/P,PSO J318.5-22,70.0,False,130.0,0.0,1247.0,0.0,3.0,Hary Waltonson
3577,7794_01,Mars,False,E/511/S,TRAPPIST-1e,39.0,False,1173.0,0.0,0.0,0.0,0.0,Newtch Meau


In [84]:
# !pip install pandas-profiling
# !pip install ydata-profiling
# from ydata_profiling import ProfileReport
# prof = ProfileReport(train)
# prof.to_file(output_file='spaceship.html')

In [85]:
train[['HomePlanet','Transported']].groupby('HomePlanet').mean()

,Transported
HomePlanet,
Earth,0.423946
Europa,0.658846
Mars,0.523024


In [86]:
train[['CryoSleep','Transported']].groupby('CryoSleep').mean() 

,Transported
CryoSleep,
False,0.328921
True,0.817583


In [87]:
train[['Destination','Transported']].groupby('Destination').mean()

,Transported
Destination,
55 Cancri e,0.610000
PSO J318.5-22,0.503769
TRAPPIST-1e,0.471175


In [88]:
train[['VIP','Transported']].groupby('VIP').mean() 

,Transported
VIP,
False,0.506332
True,0.381910


In [89]:
train = train.drop(['Name','Cabin','PassengerId'],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Destination   8511 non-null   object 
 3   Age           8514 non-null   float64
 4   VIP           8490 non-null   object 
 5   RoomService   8512 non-null   float64
 6   FoodCourt     8510 non-null   float64
 7   ShoppingMall  8485 non-null   float64
 8   Spa           8510 non-null   float64
 9   VRDeck        8505 non-null   float64
 10  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(4)
memory usage: 687.8+ KB


In [90]:
PassengerId = test['PassengerId']
test = test.drop(['Name','Cabin','PassengerId'],axis=1)
test.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4190 non-null   object 
 1   CryoSleep     4184 non-null   object 
 2   Destination   4185 non-null   object 
 3   Age           4186 non-null   float64
 4   VIP           4184 non-null   object 
 5   RoomService   4195 non-null   float64
 6   FoodCourt     4171 non-null   float64
 7   ShoppingMall  4179 non-null   float64
 8   Spa           4176 non-null   float64
 9   VRDeck        4197 non-null   float64
dtypes: float64(6), object(4)
memory usage: 334.3+ KB


In [91]:
gen_age=np.random.randint(train['Age'].mean()-train['Age'].std(),train['Age'].mean()+train['Age'].std(), size=179)
train['Age'][np.isnan(train['Age'])]=gen_age

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
gen_age=np.random.randint(test['Age'].mean()-test['Age'].std(),test['Age'].mean()+test['Age'].std(), size=91)
test['Age'][np.isnan(test['Age'])]=gen_age

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [93]:
scaler = MinMaxScaler()
train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.fit_transform(train[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
train.head()
train['Expenses'] = (train['RoomService'] + train['FoodCourt'] + train['ShoppingMall'] + train['Spa'] + train['VRDeck'])/5
train = train.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'], axis=1)
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,Expenses
0,Europa,False,TRAPPIST-1e,39.0,False,False,0.000000
1,Earth,False,TRAPPIST-1e,24.0,False,True,0.007059
2,Europa,False,TRAPPIST-1e,58.0,True,False,0.084930
3,Europa,False,TRAPPIST-1e,33.0,False,False,0.043078
4,Earth,False,TRAPPIST-1e,16.0,False,True,0.011044


In [94]:
scaler = MinMaxScaler()
test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']] = scaler.fit_transform(test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])
test.head()
test['Expenses'] = (test['RoomService'] + test['FoodCourt'] + test['ShoppingMall'] + test['Spa'] + test['VRDeck'])/5
test = test.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'], axis=1)
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,Expenses
0,Earth,True,TRAPPIST-1e,27.0,False,0.000000
1,Earth,False,TRAPPIST-1e,19.0,False,0.028523
2,Europa,True,55 Cancri e,31.0,False,0.000000
3,Europa,False,TRAPPIST-1e,38.0,False,0.059719
4,Earth,False,TRAPPIST-1e,20.0,False,0.015489


In [95]:
train['HomePlanet'].fillna('Earth', inplace=True)
train['CryoSleep'].fillna('False', inplace=True)
train['Destination'].fillna('TRAPPIST-1e',inplace=True)
train['VIP'].fillna('False',inplace=True)
train['Expenses'].fillna(train['Expenses'].mean(),inplace=True)
train['Transported'] = train['Transported'].astype(str)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   8693 non-null   object 
 1   CryoSleep    8693 non-null   object 
 2   Destination  8693 non-null   object 
 3   Age          8693 non-null   float64
 4   VIP          8693 non-null   object 
 5   Transported  8693 non-null   object 
 6   Expenses     8693 non-null   float64
dtypes: float64(2), object(5)
memory usage: 475.5+ KB


In [96]:
test['HomePlanet'].fillna('Earth', inplace=True)
test['CryoSleep'].fillna('False', inplace=True)
test['Destination'].fillna('TRAPPIST-1e',inplace=True)
test['VIP'].fillna('False',inplace=True)
test['Expenses'].fillna(test['Expenses'].mean(),inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   4277 non-null   object 
 1   CryoSleep    4277 non-null   object 
 2   Destination  4277 non-null   object 
 3   Age          4277 non-null   float64
 4   VIP          4277 non-null   object 
 5   Expenses     4277 non-null   float64
dtypes: float64(2), object(4)
memory usage: 200.6+ KB


In [97]:
# Count the number of examples for each class
class_counts = train["Transported"].value_counts()

# Print the counts for each class
print(class_counts) 

True     4378
False    4315
Name: Transported, dtype: int64


In [98]:
label = train['Transported']
features = train.drop('Transported',axis=1)
features

,HomePlanet,CryoSleep,Destination,Age,VIP,Expenses
0,Europa,False,TRAPPIST-1e,39.0,False,0.000000
1,Earth,False,TRAPPIST-1e,24.0,False,0.007059
2,Europa,False,TRAPPIST-1e,58.0,True,0.084930
3,Europa,False,TRAPPIST-1e,33.0,False,0.043078
4,Earth,False,TRAPPIST-1e,16.0,False,0.011044
...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.061023
8689,Earth,True,PSO J318.5-22,18.0,False,0.000000
8690,Earth,False,TRAPPIST-1e,26.0,False,0.015946
8691,Europa,False,55 Cancri e,32.0,False,0.036998


In [99]:
features['CryoSleep'] = features['CryoSleep'].astype(str)
features['VIP'] = features['VIP'].astype(str)
test['CryoSleep'] = test['CryoSleep'].astype(str)
test['VIP'] = test['VIP'].astype(str)

In [100]:
transformer = ColumnTransformer(
transformers = [('tnf1',OneHotEncoder(sparse=False, drop='first'),['HomePlanet','CryoSleep','Destination','VIP'])],remainder='passthrough')
transformed = transformer.fit_transform(features)
features = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
features.head()

,tnf1__HomePlanet_Europa,tnf1__HomePlanet_Mars,tnf1__CryoSleep_True,tnf1__Destination_PSO J318.5-22,tnf1__Destination_TRAPPIST-1e,tnf1__VIP_True,remainder__Age,remainder__Expenses
0,1.0,0.0,0.0,0.0,1.0,0.0,39.0,0.000000
1,0.0,0.0,0.0,0.0,1.0,0.0,24.0,0.007059
2,1.0,0.0,0.0,0.0,1.0,1.0,58.0,0.084930
3,1.0,0.0,0.0,0.0,1.0,0.0,33.0,0.043078
4,0.0,0.0,0.0,0.0,1.0,0.0,16.0,0.011044


In [101]:
transformer = ColumnTransformer(
transformers = [('tnf1',OneHotEncoder(sparse=False, drop='first'),['HomePlanet','CryoSleep','Destination','VIP'])],remainder='passthrough')
transformed = transformer.fit_transform(test)
test = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
test.head()

,tnf1__HomePlanet_Europa,tnf1__HomePlanet_Mars,tnf1__CryoSleep_True,tnf1__Destination_PSO J318.5-22,tnf1__Destination_TRAPPIST-1e,tnf1__VIP_True,remainder__Age,remainder__Expenses
0,0.0,0.0,1.0,0.0,1.0,0.0,27.0,0.000000
1,0.0,0.0,0.0,0.0,1.0,0.0,19.0,0.028523
2,1.0,0.0,1.0,0.0,0.0,0.0,31.0,0.000000
3,1.0,0.0,0.0,0.0,1.0,0.0,38.0,0.059719
4,0.0,0.0,0.0,0.0,1.0,0.0,20.0,0.015489


In [102]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
label=le.fit_transform(label)
label

array([0, 1, 0, ..., 1, 0, 1])

In [103]:
# X_train, X_test, y_train, y_test=train_test_split(features,label,test_size=0.2,random_state=42)

In [104]:
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

scaler = StandardScaler()
scaler.fit(test)
test = scaler.transform(test)

In [105]:
features

array([[ 1.754795  , -0.50366411, -0.73277002, ..., -0.15306307,
         0.70787932, -0.58901126],
       [-0.56986714, -0.50366411, -0.73277002, ..., -0.15306307,
        -0.33464345, -0.27119454],
       [ 1.754795  , -0.50366411, -0.73277002, ...,  6.53325471,
         2.02840816,  3.23450677],
       ...,
       [-0.56986714, -0.50366411, -0.73277002, ..., -0.15306307,
        -0.19564042,  0.12888528],
       [ 1.754795  , -0.50366411, -0.73277002, ..., -0.15306307,
         0.22136869,  1.07661158],
       [ 1.754795  , -0.50366411, -0.73277002, ..., -0.15306307,
         1.05538691,  0.91049382]])

In [106]:
# xgbc = XGBClassifier()

# xgbc_params = {
#         "gamma": [0.5, 1, 1.5],
#         "subsample": [0.6, 0.8, 1.0],
#         "colsample_bytree": [0.6, 0.8, 1.0],
#         "max_depth": [3, 4, 5],
#         "n_estimators": [100, 130, 150]
# }

# xgbc_cv_model = GridSearchCV(xgbc, xgbc_params, cv = 10, n_jobs = -1)
# xgbc_cv_model.fit(X_train, y_train)

# print("Best hyperparametres of the model: \n", xgbc_cv_model.best_params_)

In [107]:
xgbc_tuned = XGBClassifier(gamma = 1.5,
                           subsample = 1.0,
                           max_depth = 3,
                           colsample_bytree = 1.0,
                           n_estimators = 100)
xgbc_tuned = xgbc_tuned.fit(features, label)

In [108]:
# y_pred = xgbc_tuned.predict(X_test)
# accuracy_score(y_test, y_pred)

In [109]:
# print(classification_report(y_test, y_pred))

In [110]:
yf = xgbc_tuned.predict(test)
yf

array([1, 0, 1, ..., 1, 0, 1])

In [111]:
predictions=[]
for i in yf:
    if i==0:
        predictions.append("False")
    else:
        predictions.append("True")
predictions[0]

'True'

In [112]:
submission=pd.DataFrame()
submission['PassengerId']=PassengerId
submission['Transported']=predictions
submission.to_csv('submission.csv', index=False)

In [113]:
# grid = dict()
# grid['n_estimators'] = [10, 50, 100, 500]
# grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
# grid['algorithm'] = ['SAMME', 'SAMME.R']
# grid_search = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=grid, n_jobs=-1, cv=10, scoring='accuracy')
# # execute the grid search
# grid_result = grid_search.fit(X_train, y_train)
# # summarize the best score and configuration
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [114]:
# abc = AdaBoostClassifier(n_estimators=1500,learning_rate=0.1)
# np.mean(cross_val_score(abc,X_train,y_train,scoring='accuracy',cv=10))

In [115]:
# clf1 = LogisticRegression()
# clf2 = RandomForestClassifier()
# clf3 = KNeighborsClassifier()
# estimators = [('lr',clf1),('rf',clf2),('knn',clf3)]
# for estimator in estimators:
#     x = cross_val_score(estimator[1],X_train,y_train,cv=10,scoring='accuracy')
#     print(estimator[0],np.round(np.mean(x),2))
# vc = VotingClassifier(estimators=estimators,voting='soft')
# x = cross_val_score(vc,X_train,y_train,cv=10,scoring='accuracy')
# print(np.round(np.mean(x),2))

In [116]:
# # Instantiate an SVR model with an RBF kernel
# svr = SVR(kernel='rbf', C=1.0, gamma='scale')

# # Train the SVR model on the training set
# svr.fit(X_train, y_train)

# # Make predictions on the testing set
# y_pred = svr.predict(X_test)

# # Evaluate the performance of the SVR model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")
# print(f"R-squared: {r2}") 
# lr = LogisticRegression()
# lr.fit(X_train, y_train)
# y_pred = lr.predict(X_test)
# y_pred 
# print(classification_report(y_pred, y_test, zero_division=0))
# print(accuracy_score(y_test,y_pred)) 